In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [5]:
mitig = pd.read_csv("mitigation_cleaned.csv")
country = pd.read_csv("Google_US_only_cleaned.csv")
states_only = pd.read_csv("Google_US_and_states_cleaned.csv")

In [6]:
# convert date columns in each data set to be datetime objects
country.loc[:,'date'] = pd.to_datetime(country['date'])
states_only.loc[:,'date'] = pd.to_datetime(states_only['date'])
mitig.loc[:,'Date'] = pd.to_datetime(mitig['Date'])


In [90]:
# Find the minimum mobility value in a window after a mitigation event - US only

window_length = 7
window = np.timedelta64(window_length,'D')
google_mob_categories = country.columns[-6:] # mobility data categories


first_date = country["date"].iloc[0]
last_date = country["date"].iloc[-1]

lowest_mobility_val = []
    
# select rows in USA after earliest date  and before last date in google data from mitigation data
us_only = mitig.loc[(mitig["State"] == "USA")]
us_only = us_only.loc[(us_only["Date"] > first_date) & (us_only["Date"] < (last_date - window))]

for i,date in enumerate(us_only["Date"]):
    new_row = {"state": "USA"}
    new_row.update({"mitigation_date": date})
    new_row.update({"mitigation_type": us_only["Mitigation type"].iloc[i]})

    # get min value from each google mob category
    for cat in google_mob_categories:
        
        # get row index of min value in category for country between mitigation date and date + window
        where = country.loc[(country["date"] >= date) & (country["date"] <= date + window)][cat].idxmin()
        # add the value
        new_row.update({cat: country[cat].iloc[where]})
        # add the date of the value
        min_date = country["date"].iloc[where]
        new_row.update({"date_of_min_" + cat: min_date})
        #a add the delay
        delay = min_date - date
        new_row.update({"delay_" + cat: delay})
    lowest_mobility_val.append(new_row);
    
lowest_mobility_vals_US = DataFrame(lowest_mobility_val)
                                                   

In [89]:
lowest_mobility_vals_US

,state,mitigation_date,mitigation_type,retail_and_recreation_percent_change_from_baseline,date_of_min_retail_and_recreation_percent_change_from_baseline,delayretail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,date_of_min_grocery_and_pharmacy_percent_change_from_baseline,delaygrocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,...,delayparks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,date_of_min_transit_stations_percent_change_from_baseline,delaytransit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,date_of_min_workplaces_percent_change_from_baseline,delayworkplaces_percent_change_from_baseline,residential_percent_change_from_baseline,date_of_min_residential_percent_change_from_baseline,delayresidential_percent_change_from_baseline
0,USA,2020-02-28,other,5.0,2020-03-06,7 days,3.0,2020-02-28,0 days,10.0,...,0 days,2.0,2020-03-02,3 days,2.0,2020-02-28,0 days,-2.0,2020-02-29,1 days
1,USA,2020-02-29,movement restriction,5.0,2020-03-06,6 days,3.0,2020-03-06,6 days,10.0,...,2 days,2.0,2020-03-02,2 days,2.0,2020-03-03,3 days,-2.0,2020-02-29,0 days
2,USA,2020-03-03,other,5.0,2020-03-06,3 days,3.0,2020-03-06,3 days,12.0,...,3 days,-3.0,2020-03-10,7 days,-2.0,2020-03-10,7 days,-1.0,2020-03-03,0 days
3,USA,2020-03-06,other,2.0,2020-03-13,7 days,3.0,2020-03-06,0 days,4.0,...,7 days,-10.0,2020-03-13,7 days,-7.0,2020-03-13,7 days,-1.0,2020-03-06,0 days
4,USA,2020-03-11,movement restriction,-24.0,2020-03-18,7 days,8.0,2020-03-18,7 days,-1.0,...,5 days,-30.0,2020-03-18,7 days,-31.0,2020-03-18,7 days,0.0,2020-03-11,0 days
5,USA,2020-03-11,movement restriction,-24.0,2020-03-18,7 days,8.0,2020-03-18,7 days,-1.0,...,5 days,-30.0,2020-03-18,7 days,-31.0,2020-03-18,7 days,0.0,2020-03-11,0 days
6,USA,2020-03-13,movement restriction,-32.0,2020-03-20,7 days,5.0,2020-03-20,7 days,-2.0,...,7 days,-37.0,2020-03-20,7 days,-36.0,2020-03-20,7 days,3.0,2020-03-13,0 days
7,USA,2020-03-13,state of emergency,-32.0,2020-03-20,7 days,5.0,2020-03-20,7 days,-2.0,...,7 days,-37.0,2020-03-20,7 days,-36.0,2020-03-20,7 days,3.0,2020-03-13,0 days
8,USA,2020-03-15,movement restriction,-44.0,2020-03-22,7 days,-13.0,2020-03-22,7 days,-12.0,...,7 days,-44.0,2020-03-22,7 days,-36.0,2020-03-20,5 days,3.0,2020-03-15,0 days
9,USA,2020-03-15,events cancelled,-44.0,2020-03-22,7 days,-13.0,2020-03-22,7 days,-12.0,...,7 days,-44.0,2020-03-22,7 days,-36.0,2020-03-20,5 days,3.0,2020-03-15,0 days


In [93]:
# Find the minimum mobility value in a window after a mitigation event - States only
window_length = 7
window = np.timedelta64(window_length,'D')
google_mob_categories = states_only.columns[-6:] # mobility data categories


first_date = states_only["date"].iloc[0]
last_date = states_only["date"].iloc[-1]

lowest_mobility_val = []
states = mitig["State"].unique()
states = np.delete(states, 0) # remove 'USA' from the list

for state in states:
    
    # select rows in region after earliest date  and before last date in google data from mitigation data
    current_state = mitig.loc[(mitig["State"] == state)]
    current_state = current_state.loc[(current_state["Date"] > first_date) & (current_state["Date"] < (last_date - window))]
    
    for i,date in enumerate(current_state["Date"]):
        new_row = {"state": state}
        new_row.update({"mitigation_date": date})
        new_row.update({"mitigation_type": current_state["Mitigation type"].iloc[i]})
        
        # get min value from each google mob category
        for cat in google_mob_categories:
            
            # get row index of min value in category for states_only between mitigation date and date + window
            where = states_only.loc[(states_only["date"] >= date) & (states_only["date"] <= date + window)][cat].idxmin()
            # add the value
            new_row.update({cat: states_only[cat].iloc[where]})
            # add the date
            min_date = states_only["date"].iloc[where]
            new_row.update({"date_of_min_" + cat: min_date})
            # add the delay
            delay = min_date - date
            new_row.update({"delay_" + cat: delay})
        lowest_mobility_val.append(new_row);
    
lowest_mobility_vals_states = DataFrame(lowest_mobility_val)

In [94]:
lowest_mobility_vals_states

,state,mitigation_date,mitigation_type,retail_and_recreation_percent_change_from_baseline,date_of_min_retail_and_recreation_percent_change_from_baseline,delay_retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,date_of_min_grocery_and_pharmacy_percent_change_from_baseline,delay_grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,...,delay_parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,date_of_min_transit_stations_percent_change_from_baseline,delay_transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,date_of_min_workplaces_percent_change_from_baseline,delay_workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,date_of_min_residential_percent_change_from_baseline,delay_residential_percent_change_from_baseline
0,Alabama,2020-03-02,movement restriction,-3.0,2020-03-07,5 days,-5.0,2020-03-06,4 days,-30.0,...,3 days,-18.0,2020-03-02,0 days,-10.0,2020-03-09,7 days,-5.0,2020-03-06,4 days
1,Alabama,2020-03-06,new public services,-12.0,2020-03-13,7 days,-5.0,2020-03-06,0 days,-23.0,...,7 days,-28.0,2020-03-13,7 days,-17.0,2020-03-13,7 days,-5.0,2020-03-06,0 days
2,Alabama,2020-03-11,other,-45.0,2020-03-18,7 days,-13.0,2020-03-18,7 days,-44.0,...,3 days,-59.0,2020-03-18,7 days,-56.0,2020-03-18,7 days,-3.0,2020-03-11,0 days
3,Alabama,2020-03-12,other,-46.0,2020-03-19,7 days,-13.0,2020-03-18,6 days,-52.0,...,7 days,-63.0,2020-03-19,7 days,-58.0,2020-03-19,7 days,-2.0,2020-03-13,1 days
4,Alabama,2020-03-12,events cancelled,-46.0,2020-03-19,7 days,-13.0,2020-03-18,6 days,-52.0,...,7 days,-63.0,2020-03-19,7 days,-58.0,2020-03-19,7 days,-2.0,2020-03-13,1 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,Wyoming,2020-03-27,new public services,-68.0,2020-03-28,1 days,-42.0,2020-03-29,2 days,-69.0,...,1 days,-77.0,2020-03-31,4 days,-70.0,2020-04-01,5 days,7.0,2020-03-29,2 days
2293,Wyoming,2020-03-27,other,-68.0,2020-03-28,1 days,-42.0,2020-03-29,2 days,-69.0,...,1 days,-77.0,2020-03-31,4 days,-70.0,2020-04-01,5 days,7.0,2020-03-29,2 days
2294,Wyoming,2020-03-30,other,-67.0,2020-04-04,5 days,-34.0,2020-04-05,6 days,-70.0,...,6 days,-77.0,2020-03-31,1 days,-70.0,2020-04-01,2 days,8.0,2020-04-05,6 days
2295,Wyoming,2020-04-03,movement restriction,-67.0,2020-04-04,1 days,-36.0,2020-04-09,6 days,-74.0,...,7 days,-78.0,2020-04-07,4 days,-71.0,2020-04-07,4 days,8.0,2020-04-05,2 days
